In [1]:
#Allow Notebook to Import from Comp_Chem_Package
import sys
if("win" in sys.platform):
    sys.path.append("..\\..\\Backend")
else: 
    sys.path.append("../../Backend")
    
from nistScraper import getDiatomicConstants 
from morsePotential import morsePotential
from howPotential import howPotential
from wavefunction import wavefunction
from operators import HOperator
from schrodinger import schrod
from basisSet import basisSet
from compChemGlobal import *
from how import how
from rkr import rkr

## Harmonic Oscillator Wavefunction
<br>
The harmonic oscillator wavefunction arises from solving the Schrödinger for the harmonic oscillator. <br>
Answer the following questions on the below regarding the harmonic oscillator: 

In [2]:
from diatomicConstants import buildDiatomicConstants
diatomicSymbol = widgets.Dropdown(
    options = ["Cl2", "CO", "HCl", "HF", "O2", "OH", "HF", "N2", "NO"],
    description = "Diatomic", 
    value = "OH"
)
parameterOptions = ["Normal", "Times 2", "Divide 2"]
u = widgets.Dropdown(
    options = parameterOptions, 
    description = "μ", 
    value = "Normal"
)
w = widgets.Dropdown(
    options = parameterOptions, 
    description = "ω", 
    value = "Normal"
)
comparison = widgets.Dropdown(
    options = ["Harmonic", "Anharmonic"],
    description = "Comparison", 
    value = "Harmonic"
)
diatomic = widgets.HBox([widgets.Label("Diatomic Symbol"), diatomicSymbol, comparison, u, w])

calculate = startButton = widgets.ToggleButton(
    value = False, 
    description = "Update Solution",
    button_style = "info"
)

info = widgets.HTML("")
graph = widgets.VBox()
out = widgets.Output()
    
def inform(value):
    info.value = value
    out.clear_output()
    with out: 
        display(info)

#-------------------------------------------------------------------------------------

def buildSolution(dc, name = "", size = 20):
    pes =  howPotential(dc)
    basis = basisSet(dc, how, size)
    sol = schrod()
    sol.graphTitle = basis.diatomicConstants["name"] + " Harmonic Oscillator Schrödinger Solution"
    for index, bf in enumerate(basis):
        #eigen value analytical equation derived from hyperphysics website for harmonic osciallator
        
        ev = (index + 0.5) * dc["w"]
        evec = [1 if i == index else 0 for i in range(basis.size)]
        wf = wavefunction(evec, ev, basis, index).scale(500)
        wf2 = wavefunction(evec, ev, basis, index, squared=True).scale(500)
        wf.setGraphVariables(yEqualsCutoff = ev)
        wf2.setGraphVariables(yEqualsCutoff = ev)
        
        sol.addGraphableObject(wf)
        sol.addGraphableObject(wf2)
        
    sol.addGraphableObject(pes)
    sol.scaleFactor = 500
    sol.start = dc["re"] - 0.4
    sol.end = dc["re"] + 0.4
    sol.pesLocations.append(len(sol.graphableObjects) - 1)
    
    return sol
 
#-------------------------------------------------------------------------------------
    
def howPESUpdate():
    dc = getDiatomicConstants(diatomicSymbol.value)
    if(dc == False):
        info.value = "'" + diatomicSymbol.value + "' is not a valid diatomic molecule." 
        return
    info.value = "Calculating solution for " + diatomicSymbol.value + "..." 
    
    sol = buildSolution(dc)
    name = ""
    #update diatomic constants
    if(comparison.value == "Harmonic"):
        for widget, index in [(u, "u"), (w, "w")]:
            if(widget.value == "Times 2"):
                dc[index] *= 2
                name += ", " + widget.description + " " + widget.value
            elif(widget.value == "Divide 2"):
                dc[index] /= 2
                name += ", " + widget.description + " " + widget.value
                
        if(name != ""):
            sol.combineSolutions(buildSolution(dc, name), name)
    else: 
        sol2 = schrod()
        sol2.load(dc["name"], "Harmonic Oscillator", 100, "Morse Potential", "RKR", scaleFactor = 500)
        sol.combineSolutions(sol2, " Anharmonic")
    
    endPoints = [dc["re"] - 1.2, dc["re"] + 1.2]
    sol.start, sol.end = endPoints[0], endPoints[1]
    graph.children = ( (sol.graph(getGraph = True), diatomic, calculate, info) )
    graph.children[0].children[0].update_xaxes(range=endPoints).update_yaxes(range=[0,sol.graphableObjects[1].energy + 5000])
    
    out.clear_output()
    with out: 
        display(graph)
    
    info.value = "Calculation Completed"

updateWrapper = lambda change :  howPESUpdate()

calculate.observe(updateWrapper, names="value")

display(out)
inform("Building Graph...")
howPESUpdate()  

Output()

[0.999885, 0.000115, 0]
[1.00782503223, 2.01410177812, 3.0160492779]
[1.00782503223]
[0.99757, 0.00038, 0.00205]
[15.99491461957, 16.9991317565, 17.99915961286]
[1.00782503223, 15.99491461957]


<br><br>
Answer the below qustions using the interactive graph of the v = 0, 1 and 2 wavefunctions.

<br><br>
Using the graph above, change the mode option to "Probability Distribution", and answer the following questions in units of angstroms. 

<br>
The following questions will revolve around comparing a model of a diatomic molecule using a harmonic potential energy surface, and an anharmonic poetential energy surface. Specifically, the harmonic potential will be the haromonic oscillator presented above, while the anharmonic potential will be the anharmonic morse potential. On the graph above, change the Comnparison mode from "Harmonic" to "Anharmonic". 